In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import math
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import mouse
import cv2
import time
import pickle

In [ ]:
# It draws the skeleton no the hand.

# configure the format
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

# create function
def draw_landmarks_on_image(rgb_image, detection_result, hand):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # loop through the detected hands to visualize
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # draw the hand landmarks
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # get the top left corner of the detected hand's bounding box
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # write hand gesture on the image
    cv2.putText(annotated_image, str(hand),
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
# It prepares the training data.
import math
def normalformat(datafile):
    with open(datafile) as f: # open file and read it in
        lines = f.readlines()

    data_0 = lines[0] # prepare the list for the data
    
    # separate the data from the texts
    data_0 = data_0[2:(len(data_0)-2)]
    data_1 = data_0.split("NormalizedLandmark(")
    data_1 = data_1[1:22]
    
    # unify the data
    for i in range(19):
        data_1[i] = data_1[i][0:len(data_1[i])-3]
    data_1[20] = data_1[20][0:len(data_1[20])-1]
    
    # remove "="
    data_list = []
    for i in range(len(data_1)):
        list = data_1[i].split("=")
        data_list.append(list)
        
    # add the 21 point's coordinates
    data = []
    for i in data_list:
        list_0 = []
        list_0.append(float(i[1][0:-3]))
        list_0.append(float(i[2][0:-3]))
        list_0.append(float(i[3][0:-12]))
        data.append(list_0)
        
    # choose reference point
    ref_point = data[0]
    
    # distance from every point to every point
    difference = []
    
    for i in range(21): # because there are 21 points
        for n in range(len(data)-i):
            # x coordinate
            x = data[i][0] - data[n+i][0]
    
            # y coordinate
            y = data[i][1] - data[n+i][1]
    
            # z coordinate
            z = data[i][2] - data[n+i][2]
        
            difference.append([x, y, z])

    distance = []
    
    # see which point is lower (y < 0 == up, y >= 0 == down)
    for i in difference:
        if i[2] < 0:
            multiplier = -1
        else:
            multiplier = 1
            
        dist = math.sqrt(i[0]**2 + i[1]**2 + i[2]**2)*multiplier # multiply with -1 if the first point is lower
        distance.append(dist)
    
    # standardize the data by the distance of point 0 and 16
    unit = 1/(distance[16])
    
    # add training data to the list if data != 0
    training_data = []
    for i in distance:
        if i != -0.0:
            training_data.append(i*unit)
    
    return training_data

In [ ]:
# It calculates the average of x and y coordinates of the hand.
def coordinate(datafile):
    with open(datafile) as f: # open file and read it in
        lines = f.readlines()

    data_0 = lines[0] # prepare the list for the data
    
    # separate the data from the texts
    data_0 = data_0[2:(len(data_0)-2)]
    data_1 = data_0.split("NormalizedLandmark(")
    data_1 = data_1[1:22]
    
    # unify the data
    for i in range(19):
        data_1[i] = data_1[i][0:len(data_1[i])-3]
    data_1[20] = data_1[20][0:len(data_1[20])-1]
    
    # remove "="
    data_list = []
    for i in range(len(data_1)):
        list = data_1[i].split("=")
        data_list.append(list)
        
    # add the 21 point's coordinates
    data = []
    for i in data_list:
        list_0 = []
        list_0.append(float(i[1][0:-3]))
        list_0.append(float(i[2][0:-3]))
        list_0.append(float(i[3][0:-12]))
        data.append(list_0)
    
    # calculate average of coordinates
    x = []
    y = []
    
    for i in data:
        x.append(i[0])
        y.append(i[1])
    
    x_mean = sum(x) / len(x)
    y_mean = sum(y) / len(y)
    
    return x_mean, y_mean

In [ ]:
# Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

In [ ]:
# This is the main code. It imports the camera, recognise the hand gesture and controls the mouse.

# load the trained hand gesture recogniser model
logmodel = pd.read_pickle(r'2024_stemsisters_project.pickle')

# load camera
cam = cv2.VideoCapture(0)

# load standard coordinates for beginning
x_coord, y_coord = 0, 0

# main program
while True:
        
    # capture the video frame by frame and save the image to file
    ret, frame = cam.read()
    name = 'hand.jpg'
    cv2.imwrite(name, frame)
    
    # run the mediapipe hand landmark recogniser model on the image
    image = mp.Image.create_from_file(name)
    detection_result = detector.detect(image)
    
    # save hand landmark coordinates to file
    file_1 = open('real_time_hand.txt', 'w')
    file_1.write(str(detection_result.hand_landmarks))
    file_1.close
    
    # try to recognise hand
    try:
        # put data into dataframe format
        columns_0 = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210')
        df_0 = pd.DataFrame(normalformat('real_time_hand.txt')).T
        df_0.columns = columns_0
        hand = logmodel.predict(df_0)
        
        # hand coordinate average
        x_coord_past, y_coord_past = x_coord, y_coord
        x_coord, y_coord = coordinate('real_time_hand.txt')
        
    except:
        pass

    # process the classification result and visualize it
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result, hand)
    cv2.imshow('camera', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
    
    # scroll the page up
    if hand == ['up']:
        mouse.wheel(0.5)
    
    # scroll the page down
    if hand == ['down']:
        mouse.wheel(-0.5)
    
    # click with the left part of the mouse
    if hand == ['click']:
        mouse.click('left')
        time.sleep(1)
    
    # move mouse if the hand is moving
    if hand == ['mouse']:
        mouse.move(int((x_coord-x_coord_past) * -500), int((y_coord-y_coord_past) * 500), absolute=False, duration=0)
    

    # if "q" is pressed the program stops

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

# after the loop release the cap object 
cam.release() 

# destroy all the windows 
cv2.destroyAllWindows()